#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import users table:

In [3]:
users = pd.read_csv('users_table.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [4]:
users.rename(columns= {'Id': 'userId'}, inplace= True)

#### 4. Import posts table:

In [5]:
posts = pd.read_csv('posts_table.csv')
posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [6]:
posts.rename(columns= {'Id': 'postId', 'OwnerUserId':'userId'}, inplace= True)

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [7]:
users_new = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts_new = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]


new_dataframe = pd.concat([users_new, posts_new], axis=1)
new_dataframe.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userId,ViewCount,CommentCount
0,-1,1,0,5007,1920,1.0,23.0,8.0,1278.0,1.0
1,2,101,25,3,0,2.0,22.0,24.0,8198.0,1.0
2,3,101,22,19,0,3.0,54.0,18.0,3613.0,4.0
3,4,101,11,0,0,4.0,13.0,23.0,5224.0,2.0
4,5,6792,1145,662,5,5.0,81.0,23.0,NaN,3.0


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [8]:
merged = users.merge(posts)
merged

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,-1.0,2014-04-23 13:43:43,NaN,NaN,NaN,NaN,NaN
1,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,-1.0,2011-03-21 17:40:28,NaN,NaN,NaN,NaN,NaN
2,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,-1.0,2011-03-21 17:46:43,NaN,NaN,NaN,NaN,NaN
3,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,919.0,2011-03-30 19:23:14,NaN,NaN,NaN,NaN,NaN
4,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,919.0,2011-03-30 19:23:14,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38957,45934,11,2014-05-21 17:13:23,jasonweiyi,2014-06-08 13:36:47,NaN,NaN,NaN,1,0,...,1.0,2,NaN,805.0,2013-05-13 04:41:41,NaN,NaN,NaN,jasonweiyi,NaN
38958,46192,36,2014-05-26 15:29:30,user1738753,2014-09-10 19:52:34,NaN,NaN,NaN,1,0,...,1.0,2,1.0,930.0,2012-10-18 15:07:40,NaN,NaN,NaN,user1738753,NaN
38959,46522,235,2014-06-01 17:25:18,Andy Blankertz,2014-09-13 18:03:07,NaN,NaN,<p>Actuary for a life insurance company.</p>\r\n,13,27,...,3.0,0,NaN,NaN,NaN,NaN,NaN,NaN,user1009703,NaN
38960,52371,221,2014-07-19 13:36:58,Karel Petranek,2014-07-20 09:32:16,NaN,NaN,NaN,2,0,...,7.0,5,8.0,NaN,NaN,2014-07-30 20:09:14,NaN,NaN,dark_charlie,NaN


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [9]:
ds_null = merged.isnull().sum()
ds_null[ds_null > 0]

DisplayName                  9
WebsiteUrl               21496
Location                 18057
AboutMe                  18488
Age                      24862
ProfileImageUrl          36137
AcceptedAnswerId         31813
ViewCount                23572
Body                       115
Title                    23572
Tags                     23572
AnswerCount              23572
FavoriteCount            31906
LastEditorUserId         19624
LastEditDate             19434
CommunityOwnedDate       37136
ParentId                 15923
ClosedDate               38451
OwnerDisplayName         38224
LastEditorDisplayName    38753
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [22]:
# change missing values with 0
merged.fillna(0)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,0.0,0,0.0,-1.0,2014-04-23 13:43:43,0,0.0,0,0,0
1,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,0.0,0,0.0,-1.0,2011-03-21 17:40:28,0,0.0,0,0,0
2,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,0.0,0,0.0,-1.0,2011-03-21 17:46:43,0,0.0,0,0,0
3,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,0.0,0,0.0,919.0,2011-03-30 19:23:14,0,0.0,0,0,0
4,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,0.0,0,0.0,919.0,2011-03-30 19:23:14,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38957,45934,11,2014-05-21 17:13:23,jasonweiyi,2014-06-08 13:36:47,0,0,0,1,0,...,1.0,2,0.0,805.0,2013-05-13 04:41:41,0,0.0,0,jasonweiyi,0
38958,46192,36,2014-05-26 15:29:30,user1738753,2014-09-10 19:52:34,0,0,0,1,0,...,1.0,2,1.0,930.0,2012-10-18 15:07:40,0,0.0,0,user1738753,0
38959,46522,235,2014-06-01 17:25:18,Andy Blankertz,2014-09-13 18:03:07,0,0,<p>Actuary for a life insurance company.</p>\r\n,13,27,...,3.0,0,0.0,0.0,0,0,0.0,0,user1009703,0
38960,52371,221,2014-07-19 13:36:58,Karel Petranek,2014-07-20 09:32:16,0,0,0,2,0,...,7.0,5,8.0,0.0,0,2014-07-30 20:09:14,0.0,0,dark_charlie,0


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [19]:
merged.dtypes

userId                     int64
Reputation                 int64
CreationDate              object
DisplayName               object
LastAccessDate            object
WebsiteUrl                object
Location                  object
AboutMe                   object
Views                      int64
UpVotes                    int64
DownVotes                  int64
AccountId                  int64
Age                      float64
ProfileImageUrl           object
postId                     int64
PostTypeId                 int64
AcceptedAnswerId         float64
CreaionDate               object
Score                      int64
ViewCount                float64
Body                      object
LasActivityDate           object
Title                     object
Tags                      object
AnswerCount              float64
CommentCount               int64
FavoriteCount            float64
LastEditorUserId         float64
LastEditDate              object
CommunityOwnedDate        object
ParentId  

In [33]:
# changed date columns to datatime type
merged.astype({'CreationDate': 'datetime64', 'LastAccessDate': 'datetime64', 'LasActivityDate': 'datetime64', 'ClosedDate': 'datetime64',
                'CreaionDate': 'datetime64', 'LastEditDate': 'datetime64'}).dtypes

userId                            int64
Reputation                        int64
CreationDate             datetime64[ns]
DisplayName                      object
LastAccessDate           datetime64[ns]
WebsiteUrl                       object
Location                         object
AboutMe                          object
Views                             int64
UpVotes                           int64
DownVotes                         int64
AccountId                         int64
Age                             float64
ProfileImageUrl                  object
postId                            int64
PostTypeId                        int64
AcceptedAnswerId                float64
CreaionDate              datetime64[ns]
Score                             int64
ViewCount                       float64
Body                             object
LasActivityDate          datetime64[ns]
Title                            object
Tags                             object
AnswerCount                     float64
